<div style="width: 100%; overflow: hidden;">
    <div style="width: 150px; float: left;"> <img src="data/D4Sci_logo_ball.png" alt="Data For Science, Inc" align="left" border="0"> </div>
    <div style="float: left; margin-left: 10px;"> <h1>Transforming Excel Analysis into pandas Data Models</h1>
<h1>Excel Pitfalls</h1>
        <p>Bruno Gonçalves<br/>
        <a href="http://www.data4sci.com/">www.data4sci.com</a><br/>
            @bgoncalves, @data4sci</p></div>
</div>

In [1]:
from collections import Counter
from pprint import pprint

import pandas as pd
import numpy as np
import numpy_financial as npf

import matplotlib
import matplotlib.pyplot as plt 

import watermark

%load_ext watermark
%matplotlib inline

We start by print out the versions of the libraries we're using for future reference

In [2]:
%watermark -n -v -m -g -iv

json            2.0.9
watermark       2.0.2
pandas          1.0.1
numpy           1.18.1
matplotlib      3.1.3
numpy_financial 1.0.0
autopep8        1.5
Thu Sep 03 2020 

CPython 3.7.3
IPython 6.2.1

compiler   : Clang 4.0.1 (tags/RELEASE_401/final)
system     : Darwin
release    : 19.6.0
machine    : x86_64
processor  : i386
CPU cores  : 8
interpreter: 64bit
Git hash   : 645675cceecd8c43030193fc309fa409552f669e


Load default figure style

In [3]:
plt.style.use('./d4sci.mplstyle')

## Large file

Data Frames are limited only by available memory, and have no fixed limit on the number of rows or columns

In [4]:
taxis = pd.read_csv('data/green_tripdata_2014-04.csv.gz', 
        parse_dates=['lpep_pickup_datetime', 'Lpep_dropoff_datetime']
                   )

Even relatively small files can have a large number of rows

In [5]:
taxis.shape

(1309155, 22)

And we can be sure that each column has a unique format without any unexpected changes in formatting

In [6]:
taxis.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1309155 entries, 0 to 1309154
Data columns (total 22 columns):
 #   Column                 Non-Null Count    Dtype         
---  ------                 --------------    -----         
 0   VendorID               1309155 non-null  int64         
 1   lpep_pickup_datetime   1309155 non-null  datetime64[ns]
 2   Lpep_dropoff_datetime  1309155 non-null  datetime64[ns]
 3   Store_and_fwd_flag     1309155 non-null  object        
 4   RateCodeID             1309155 non-null  int64         
 5   Pickup_longitude       1309155 non-null  float64       
 6   Pickup_latitude        1309155 non-null  float64       
 7   Dropoff_longitude      1309155 non-null  float64       
 8   Dropoff_latitude       1309155 non-null  float64       
 9   Passenger_count        1309155 non-null  int64         
 10  Trip_distance          1309155 non-null  float64       
 11  Fare_amount            1309155 non-null  float64       
 12  Extra                  13091

Computations are column based and written in a compact form

In [6]:
taxis['Trip_type'].unique()

array([ 1., nan,  2.])

In [7]:
taxis['Fare_amount'].mean()

12.339962212266684

Easily index any part of the full DataFrame

In [9]:
taxis.iloc[1000:1020]

,VendorID,lpep_pickup_datetime,Lpep_dropoff_datetime,Store_and_fwd_flag,RateCodeID,Pickup_longitude,Pickup_latitude,Dropoff_longitude,Dropoff_latitude,Passenger_count,...,Extra,MTA_tax,Tip_amount,Tolls_amount,Ehail_fee,Total_amount,Payment_type,Trip_type,Unnamed: 20,
1000,2,2014-04-01 01:00:44,2014-04-01 01:13:30,N,1,-73.891830,40.747021,-73.917595,40.834263,5,...,0.5,0.5,0.0,5.33,NaN,28.33,2,1.0,NaN,NaN
1001,1,2014-04-01 01:00:45,2014-04-01 01:08:54,N,1,-73.918846,40.759144,-73.944969,40.753735,1,...,0.5,0.5,2.0,0.00,NaN,11.50,1,NaN,NaN,NaN
1002,2,2014-04-01 01:00:46,2014-04-01 01:13:03,N,5,-73.919205,40.815300,-73.946342,40.797829,1,...,0.0,0.0,0.0,0.00,NaN,0.12,2,2.0,NaN,NaN
1003,2,2014-04-01 01:00:59,2014-04-01 01:18:16,N,1,-73.937576,40.758251,-73.958313,40.731274,1,...,0.5,0.5,2.0,0.00,NaN,21.50,1,1.0,NaN,NaN
1004,2,2014-04-01 01:01:07,2014-04-01 01:01:24,N,5,-73.871689,40.852695,-73.871452,40.846371,1,...,0.0,0.0,4.0,0.00,NaN,24.00,1,2.0,NaN,NaN
1005,2,2014-04-01 01:01:07,2014-04-01 01:06:28,N,1,-73.947624,40.711250,-73.934898,40.703197,1,...,0.5,0.5,0.0,0.00,NaN,7.50,2,1.0,NaN,NaN
1006,1,2014-04-01 01:01:08,2014-04-01 01:07:29,N,1,-73.997536,40.594048,-73.998772,40.577499,1,...,0.5,0.5,0.0,0.00,NaN,9.00,2,NaN,NaN,NaN
1007,1,2014-04-01 01:01:15,2014-04-01 01:21:49,N,1,-73.975716,40.687069,-73.974289,40.646133,1,...,0.5,0.5,0.0,0.00,NaN,19.00,2,NaN,NaN,NaN
1008,2,2014-04-01 01:01:19,2014-04-01 01:10:20,N,1,-73.844254,40.721008,-73.806267,40.678631,1,...,0.5,0.5,0.0,0.00,NaN,14.50,1,1.0,NaN,NaN
1009,2,2014-04-01 01:01:22,2014-04-01 01:03:55,N,1,-73.931755,40.765072,-73.918663,40.758862,1,...,0.5,0.5,0.0,0.00,NaN,5.00,2,1.0,NaN,NaN


## Mortgage Calculator

The logic underlying any computation is always clear, as each cell displays the code instead of just the computed values

In [10]:
price = 1_000_000
down = .2 * price
ammount = price-down
years = 30
months = 12*years
rate = .03/12

In [11]:
payment = npf.pmt(rate, months, -ammount)

In [13]:
dates = pd.date_range('08/01/2020', periods=months, freq='M')+pd.Timedelta('1D')

From these few lines of code we can easily audit how the computation is performing and find any existing bugs

In [14]:
rows = []

balance = price - down 
extra_payment = 100

for month in dates:
    row = [month]
    row.append(balance)
    row.append(payment)
    row.append(extra_payment)
    row.append(payment+extra_payment)
    
    interest = balance*rate
    principal = payment-interest
    
    row.append(principal+extra_payment)
    row.append(interest)
    
    balance -= principal+extra_payment
    
    row.append(balance)
    rows.append(row)
    
    if balance <= 0:
        break

And convert the data into a compact DataFrame that can be used to perform further computations

In [15]:
mortgage = pd.DataFrame(rows, columns=['Date', 'Beginning Balance', 'Scheduled Payment', 'Extra Payment', 
                                       'Total Payment', 'Principal', 'Interest', 'Ending Balance'])
mortgage['Cummulative Interest'] = mortgage.Interest.cumsum()

In [16]:
mortgage

,Date,Beginning Balance,Scheduled Payment,Extra Payment,Total Payment,Principal,Interest,Ending Balance,Cummulative Interest
0,2020-09-01,800000.000000,3372.83227,100,3472.83227,1472.832270,2000.000000,798527.167730,2000.000000
1,2020-10-01,798527.167730,3372.83227,100,3472.83227,1476.514351,1996.317919,797050.653380,3996.317919
2,2020-11-01,797050.653380,3372.83227,100,3472.83227,1480.205636,1992.626633,795570.447743,5988.944553
3,2020-12-01,795570.447743,3372.83227,100,3472.83227,1483.906150,1988.926119,794086.541593,7977.870672
4,2021-01-01,794086.541593,3372.83227,100,3472.83227,1487.615916,1985.216354,792598.925677,9963.087026
...,...,...,...,...,...,...,...,...,...
339,2048-12-01,15664.923674,3372.83227,100,3472.83227,3433.669961,39.162309,12231.253713,392994.225458
340,2049-01-01,12231.253713,3372.83227,100,3472.83227,3442.254136,30.578134,8788.999578,393024.803592
341,2049-02-01,8788.999578,3372.83227,100,3472.83227,3450.859771,21.972499,5338.139807,393046.776091
342,2049-03-01,5338.139807,3372.83227,100,3472.83227,3459.486920,13.345350,1878.652887,393060.121440


## Non-standardized data

In [17]:
movies = pd.read_excel('data/movies.xlsx')

Unexpected spaces in the data

In [18]:
movies['Title'].iloc[0]

"Intolerance: Love's Struggle Throughout the Ages\xa0"

That can easily be cleaned

In [19]:
movies['Title'] = movies['Title'].transform(lambda x: x.strip())

In [20]:
movies['Title'].iloc[0]

"Intolerance: Love's Struggle Throughout the Ages"

<div style="width: 100%; overflow: hidden;">
     <img src="data/D4Sci_logo_full.png" alt="Data For Science, Inc" align="center" border="0" width=300px> 
</div>